# CPSC 330 - Applied Machine Learning 

## Homework 7: Word embeddings and topic modeling 
**Due date: [Apr 2, 11:59pm](https://github.com/UBC-CS/cpsc330-2023W2?tab=readme-ov-file#deliverable-due-dates-tentative).**

## Imports

In [1]:
import os
from hashlib import sha1

%matplotlib inline
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
from sklearn.feature_extraction.text import CountVectorizer

from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split
from sklearn.pipeline import Pipeline, make_pipeline

<br><br>

## Submission instructions
<hr>

_points: 2_

You will receive marks for correctly submitting this assignment. To submit this assignment, follow the instructions below:

- **You may work on this assignment in a group (group size <= 4) and submit your assignment as a group.** 
- Below are some instructions on working as a group.  
    - The maximum group size is 4. 
    - You can choose your own group members. 
    - Use group work as an opportunity to collaborate and learn new things from each other. 
    - Be respectful to each other and make sure you understand all the concepts in the assignment well. 
    - It's your responsibility to make sure that the assignment is submitted by one of the group members before the deadline. [Here](https://help.gradescope.com/article/m5qz2xsnjy-student-add-group-members) are some instructions on adding group members in Gradescope.  
- Be sure to follow the [homework submission instructions](https://github.com/UBC-CS/cpsc330-2023W2/blob/main/docs/homework_instructions.md).
- Upload the .ipynb file to PrairieLearn.

<br><br><br><br>

## Exercise 1:  Exploring pre-trained word embeddings <a name="1"></a>
<hr>

In lecture 17, we talked about natural language processing (NLP). Using pre-trained word embeddings is very common in NLP. It has been shown that pre-trained word embeddings work well on a variety of text classification tasks. These embeddings are created by training a model like Word2Vec on a huge corpus of text such as a dump of Wikipedia or a dump of the web crawl. 

A number of pre-trained word embeddings are available out there. Some popular ones are: 

- [GloVe](https://nlp.stanford.edu/projects/glove/)
    * trained using [the GloVe algorithm](https://nlp.stanford.edu/pubs/glove.pdf) 
    * published by Stanford University 
- [fastText pre-trained embeddings for 294 languages](https://fasttext.cc/docs/en/pretrained-vectors.html) 
    * trained using the fastText algorithm
    * published by Facebook
    
In this exercise, you will be exploring GloVe Wikipedia pre-trained embeddings. The code below loads the word vectors trained on Wikipedia using an algorithm called Glove. You'll need `gensim` package in your cpsc330 conda environment to run the code below. 

```
> conda activate cpsc330
> conda install -c anaconda gensim
```

In [2]:
import gensim
import gensim.downloader

print(list(gensim.downloader.info()["models"].keys()))

['fasttext-wiki-news-subwords-300', 'conceptnet-numberbatch-17-06-300', 'word2vec-ruscorpora-300', 'word2vec-google-news-300', 'glove-wiki-gigaword-50', 'glove-wiki-gigaword-100', 'glove-wiki-gigaword-200', 'glove-wiki-gigaword-300', 'glove-twitter-25', 'glove-twitter-50', 'glove-twitter-100', 'glove-twitter-200', '__testing_word2vec-matrix-synopsis']


In [3]:
# This will take a while to run when you run it for the first time.
import gensim.downloader as api

glove_wiki_vectors = api.load("glove-wiki-gigaword-100")

[==================================================] 100.0% 128.1/128.1MB downloaded


In [4]:
len(glove_wiki_vectors)

400000

There are 400,000 word vectors in this pre-trained model. 

Now that we have GloVe Wiki vectors loaded in `glove_wiki_vectors`, let's explore the embeddings. 

<br><br>

<!-- BEGIN QUESTION -->

### 1.1 Word similarity using pre-trained embeddings

_Points:_ 2

**Your tasks:**

- Come up with a list of 4 words of your choice and find similar words to these words in `glove_wiki_vectors` embeddings. 

<div class="alert alert-warning">

Solution_1.1
    
</div>

In [5]:
glove_wiki_vectors.most_similar("trash")

[('garbage', 0.9249536991119385),
 ('rubbish', 0.8067941069602966),
 ('dump', 0.7270699143409729),
 ('pile', 0.7196474075317383),
 ('waste', 0.7033370137214661),
 ('piles', 0.6982601284980774),
 ('bins', 0.6639981269836426),
 ('recycling', 0.6520524621009827),
 ('cans', 0.6504139304161072),
 ('heap', 0.6450256705284119)]

In [6]:
glove_wiki_vectors.most_similar("basket")

[('baskets', 0.7103695869445801),
 ('layup', 0.6246376633644104),
 ('ball', 0.6069576144218445),
 ('throws', 0.6052566766738892),
 ('3-pointer', 0.5942085981369019),
 ('dunk', 0.5935219526290894),
 ('clutch', 0.5659646987915039),
 ('fouled', 0.5471230149269104),
 ('3-point', 0.5469396114349365),
 ('rebound', 0.5314509272575378)]

In [7]:
glove_wiki_vectors.most_similar("sunshine")

[('sunny', 0.6617124676704407),
 ('warm', 0.5916898846626282),
 ('rain', 0.5708038806915283),
 ('chilly', 0.5552759170532227),
 ('sun', 0.5513474941253662),
 ('rainy', 0.5475090742111206),
 ('skies', 0.5463058948516846),
 ('warmth', 0.5366362929344177),
 ('cool', 0.5352686047554016),
 ('snow', 0.5339584350585938)]

In [8]:
glove_wiki_vectors.most_similar("red")

[('yellow', 0.8597684502601624),
 ('blue', 0.8435065150260925),
 ('green', 0.8204779028892517),
 ('black', 0.7679567337036133),
 ('white', 0.7675364017486572),
 ('purple', 0.7657994627952576),
 ('pink', 0.7491969466209412),
 ('orange', 0.7140781283378601),
 ('dark', 0.699396014213562),
 ('gray', 0.6913918256759644)]

<!-- END QUESTION -->

<br><br>

### 1.2 Word similarity using pre-trained embeddings

_Points:_ 2

**Your tasks:**
1. Calculate cosine similarity for the following word pairs (`word_pairs`) using the [`similarity`](https://radimrehurek.com/gensim/models/keyedvectors.html?highlight=similarity#gensim.models.keyedvectors.KeyedVectors.similarity) method of `glove_wiki_vectors`.

In [9]:
word_pairs = [
    ("coast", "shore"),
    ("clothes", "closet"),
    ("old", "new"),
    ("smart", "intelligent"),
    ("dog", "cat"),
    ("tree", "lawyer"),
]

<div class="alert alert-warning">

Solution_1.2
    
</div>

In [10]:
def get_sim(word_pair):
    return glove_wiki_vectors.similarity(word_pair[0], word_pair[1])

In [22]:
coast_shore_similarity = get_sim(word_pairs[0])
clothes_closet_similarity = get_sim(word_pairs[1])
old_new_similarity = get_sim(word_pairs[2])
smart_intelligent_similarity = get_sim(word_pairs[3])
dog_cat_similarity = get_sim(word_pairs[4])
tree_lawyer_similarity = get_sim(word_pairs[5])

print("coast_shore_similarity = ", coast_shore_similarity)
print("clothes_closet_similarity = ", clothes_closet_similarity)
print("old_new_similarity = " , old_new_similarity)
print("smart_intelligent_similarity = " , smart_intelligent_similarity)
print("dog_cat_similarity = " , dog_cat_similarity)
print("tree_lawyer_similarity = " , tree_lawyer_similarity)

coast_shore_similarity =  0.7000272
clothes_closet_similarity =  0.546276
old_new_similarity =  0.6432488
smart_intelligent_similarity =  0.7552732
dog_cat_similarity =  0.8798075
tree_lawyer_similarity =  0.076719455


<br><br>

<!-- BEGIN QUESTION -->

### 1.3 Stereotypes and biases in embeddings

_Points:_ 6

Word vectors contain lots of useful information. But they also contain stereotypes and biases of the texts they were trained on. In the lecture, we saw an example of gender bias in Google News word embeddings. Here we are using pre-trained embeddings trained on Wikipedia data. 

**Your tasks:**

1. Explore whether there are any worrisome biases or stereotypes present in these embeddings or not by trying out at least 4 examples. You can use the following two methods or other methods of your choice to explore what kind of stereotypes and biases are encoded in these embeddings. 
    - the `analogy` function below which gives word analogies (an example shown below)
    - [similarity](https://radimrehurek.com/gensim/models/keyedvectors.html?highlight=similarity#gensim.models.keyedvectors.KeyedVectors.similarity) or [distance](https://radimrehurek.com/gensim/models/keyedvectors.html?highlight=distance#gensim.models.keyedvectors.KeyedVectors.distances) methods (an example is shown below)   
2. Discuss your observations.

> Note that most of the recent embeddings are de-biased. But you might still observe some biases in them. Also, not all stereotypes present in pre-trained embeddings are necessarily bad. But you should be aware of them when you use them in your models. 

An example of using analogy to explore biases and stereotypes.

In [12]:
def analogy(word1, word2, word3, model=glove_wiki_vectors):
    """
    Returns analogy word using the given model.

    Parameters
    --------------
    word1 : (str)
        word1 in the analogy relation
    word2 : (str)
        word2 in the analogy relation
    word3 : (str)
        word3 in the analogy relation
    model :
        word embedding model

    Returns
    ---------------
        pd.dataframe
    """
    print("%s : %s :: %s : ?" % (word1, word2, word3))
    sim_words = model.most_similar(positive=[word3, word2], negative=[word1])
    return pd.DataFrame(sim_words, columns=["Analogy word", "Score"])

In [13]:
analogy("man", "doctor", "woman")

man : doctor :: woman : ?


,Analogy word,Score
0,nurse,0.773523
1,physician,0.718943
2,doctors,0.682433
3,patient,0.675068
4,dentist,0.672603
5,pregnant,0.664246
6,medical,0.652045
7,nursing,0.645348
8,mother,0.639333
9,hospital,0.638750


An example of using similarity between words to explore biases and stereotypes.  

In [14]:
glove_wiki_vectors.similarity("white", "rich")

0.447236

In [15]:
glove_wiki_vectors.similarity("black", "rich")

0.5174519

<div class="alert alert-warning">

Solution_1.3
    
</div>

In [38]:
glove_wiki_vectors.similarity("man", "smart")

0.4778229

In [39]:
glove_wiki_vectors.similarity("woman", "smart")

0.36801586

Comparison 1
- we can see that the similarity betweeen "man" and "smart" is much higher than between "woman" and "smart", highlighting a gender bias in the data.

<br>

In [51]:
glove_wiki_vectors.similarity("america", "evil")

0.38604662

In [52]:
glove_wiki_vectors.similarity("china", "evil")

0.27105933

Comparison 2
- we can see that the similarity betweeen "america" and "evil" is much higher than between "china" and "evil", highlighting a nationaly-based bias in the data.

<br>


In [54]:
glove_wiki_vectors.similarity("black", "criminal")

0.3512528

In [55]:
glove_wiki_vectors.similarity("white", "criminal")

0.3492821

In [53]:
analogy("black", "criminal", "white")

black : criminal :: white : ?


,Analogy word,Score
0,prosecution,0.765332
1,investigation,0.738572
2,indictment,0.711792
3,crimes,0.708782
4,charges,0.705500
5,prosecutors,0.697134
6,investigations,0.696814
7,case,0.679391
8,fbi,0.664048
9,justice,0.664002


In [57]:
analogy("white", "judge", "black")

white : judge :: black : ?


,Analogy word,Score
0,court,0.776896
1,judges,0.742113
2,jury,0.707071
3,supreme,0.685860
4,appeals,0.680003
5,trial,0.634514
6,verdict,0.633684
7,appeal,0.631709
8,defendant,0.629520
9,attorney,0.628437


Comparison 3
- we can see that the similarity betweeen "white" and "criminal" is approximately the same as "black" and "criminal". Together with the results from the analogy methods, this indicates that there is likely a low level of bias when considering race and crime.

<br>

In [93]:
analogy("fat", "ugly", "skinny")

fat : ugly :: skinny : ?


,Analogy word,Score
0,nasty,0.586500
1,cute,0.571785
2,awkward,0.571268
3,adorable,0.554518
4,sexy,0.552583
5,gorgeous,0.547123
6,blond,0.543462
7,pointy,0.537222
8,gangly,0.530221
9,ragged,0.529547


In [77]:
glove_wiki_vectors.similarity("fat", "gross")

0.22042097

In [76]:
glove_wiki_vectors.similarity("skinny", "gross")

-0.06481503

In [78]:
glove_wiki_vectors.similarity("fat", "pretty")

0.3780058

In [79]:
glove_wiki_vectors.similarity("skinny", "pretty")

0.49939656

Comparison 4
- The similarity scores for "fat" vs "skinny" with the words "gross" and "pretty" display a clear fat phobia/ weight bias. The word "fat" gets a higher score in relation to "gross", and the word "skinny" has a high score in relation to "pretty". The analogy method further highlights this bias, with several of the analogy words being positive in relation to "skinny".

<br>


In [105]:
barista = glove_wiki_vectors.similarity("barista", "stupid")
clerk = glove_wiki_vectors.similarity("clerk", "stupid")
plumber = glove_wiki_vectors.similarity("plumber", "stupid")
chef = glove_wiki_vectors.similarity("chef", "stupid")
waiter = glove_wiki_vectors.similarity("waiter", "stupid")
teacher = glove_wiki_vectors.similarity("teacher", "stupid")
pilot = glove_wiki_vectors.similarity("pilot", "stupid")
lawyer = glove_wiki_vectors.similarity("lawyer", "stupid")
doctor = glove_wiki_vectors.similarity("doctor", "stupid")
prof = glove_wiki_vectors.similarity("professor", "stupid")
acc = glove_wiki_vectors.similarity("accountant", "stupid")

print("similarity to the word 'stupid' : ")
print("barista: ", barista)
print("clerk: ", clerk)
print("plumber: ", plumber)
print("chef: ", chef)
print("waiter: ", waiter)
print("teacher: ", teacher)
print("pilot: ", pilot)
print("lawyer: ", lawyer)
print("doctor: ", doctor)
print("professor: ", prof)
print("accountant: ", acc)


similarity to the word 'stupid' : 
barista:  -0.10206677
clerk:  0.13321412
plumber:  0.21871105
chef:  0.12953426
waiter:  0.27270436
teacher:  0.17423308
pilot:  0.15247726
lawyer:  0.1495709
doctor:  0.21584412
professor:  0.07812043
accountant:  0.11629474


Comparison 5
- comparing a variety of job titles with the word "stupid", we see that there doesn't seem to be much consistent bias between intelligence and job title


<!-- END QUESTION -->

<br><br>

<!-- BEGIN QUESTION -->

### 1.4 Discussion

_Points:_ 4

**Your tasks:**
1. Based on your exploration above, comment on the overall quality of these pre-trained embeddings. 
2. In the lecture, we saw that our pre-trained word embedding model output an analogy that reinforced a gender stereotype. Give an example of how using such a model could cause harm in the real world.

<div class="alert alert-warning">

Solution_1.4
    
</div>

1. There seem to be some biases in the pre-trained embeddings as can be seen by the above analyses. Certain words are more heavily correlated with good or bad qualities, indicating biases in the data. However, its important to note that doing analysis such as these on single word pair comparisons may not provide the entire picture when it comes to bias in the data. As seen with Comparison 5 above, certain jobs are more correlated with the word "stupid" than others. If we had compared the similarity between "stupid" and "plumber" with the similarity between "stupid" and "professor", we may have assumed there was bias in the data, but taking a look at several examples shows that there isn't actually a consistent bias.

2. Models like this can cause harm in the real world when their biases are used in ways which unequally harm different groups of people. Consider a university that uses such a model to quickly evaluate the hundreds of applications they receive. If certain countries are associated with good or bad qualities with regards to an individual's potential success at university, and this is used to filter applications out, it could  negatively impact certain groups of people. As seen below, Eygpt and Iran have relatively low scores for "successful" and Canada, Ireland, and Russia all have relatively low scores for "smart". If any of these relationships systematically discriminate against people from certain countries, this would cause real world harm.

In [108]:
chinaf = glove_wiki_vectors.similarity("china", "failure")
indiaf = glove_wiki_vectors.similarity("india", "failure")
irelandf = glove_wiki_vectors.similarity("ireland", "failure")
russiaf = glove_wiki_vectors.similarity("russia", "failure")
americaf = glove_wiki_vectors.similarity("america", "failure")
canadaf = glove_wiki_vectors.similarity("canada", "failure")
egyptf = glove_wiki_vectors.similarity("egypt", "failure")
iranf = glove_wiki_vectors.similarity("iran", "failure")

chinas = glove_wiki_vectors.similarity("china", "smart")
indias = glove_wiki_vectors.similarity("india", "smart")
irelands = glove_wiki_vectors.similarity("ireland", "smart")
russias = glove_wiki_vectors.similarity("russia", "smart")
americas = glove_wiki_vectors.similarity("america", "smart")
canadas = glove_wiki_vectors.similarity("canada", "smart")
egypts = glove_wiki_vectors.similarity("egypt", "smart")
irans = glove_wiki_vectors.similarity("iran", "smart")

chinasu = glove_wiki_vectors.similarity("china", "successful")
indiasu = glove_wiki_vectors.similarity("india", "successful")
irelandsu = glove_wiki_vectors.similarity("ireland", "successful")
russiasu = glove_wiki_vectors.similarity("russia", "successful")
americasu = glove_wiki_vectors.similarity("america", "successful")
canadasu = glove_wiki_vectors.similarity("canada", "successful")
egyptsu = glove_wiki_vectors.similarity("egypt", "successful")
iransu = glove_wiki_vectors.similarity("iran", "successful")

similarity_scores = {
    'Country': ['China', 'India', 'Ireland', 'Russia', 'America', 'Canada', 'Egypt', 'Iran'],
    'Similarity_Failure': [chinaf, indiaf, irelandf, russiaf, americaf, canadaf, egyptf, iranf],
    'Similarity_Smart': [chinas, indias, irelands, russias, americas, canadas, egyptf, iranf],
    'Similarity_Successful': [chinasu, indiasu, irelandsu, russiasu, americasu, canadasu, egyptsu, iransu]
}

df = pd.DataFrame(similarity_scores)
df


,Country,Similarity_Failure,Similarity_Smart,Similarity_Successful
0,China,0.407563,0.269794,0.429247
1,India,0.332338,0.215138,0.384985
2,Ireland,0.328921,0.146819,0.381652
3,Russia,0.408797,0.145354,0.339599
4,America,0.404370,0.349322,0.486080
5,Canada,0.280339,0.175802,0.402819
6,Egypt,0.336232,0.336232,0.293183
7,Iran,0.420348,0.420348,0.294948


<!-- END QUESTION -->

<br><br>

### 1.5 Classification with pre-trained embeddings 

_Points:_ 8

In lecture 17, we saw that you can conveniently get word vectors with `spaCy` with `en_core_web_md` model. In this exercise, you'll use word embeddings in multi-class text classification task. We will use [HappyDB](https://www.kaggle.com/ritresearch/happydb) corpus which contains about 100,000 happy moments classified into 7 categories: *affection, exercise, bonding, nature, leisure, achievement, enjoy_the_moment*. The data was crowd-sourced via [Amazon Mechanical Turk](https://www.mturk.com/). The ground truth label is not available for all examples, and in this lab, we'll only use the examples where ground truth is available (~15,000 examples). 

- Download the data from [here](https://www.kaggle.com/ritresearch/happydb).
- Unzip the file and copy it under the data/ directory in this homework directory.

The code below reads the data CSV (assuming that it's present in the current directory as *cleaned_hm.csv*),  cleans it up a bit, and splits it into train and test splits. 

**Your tasks:**

1. Train logistic regression with bag-of-words features (`CountVectorizer`, be sure to filter out stop words), calculate the train and test accuracy and show a [classification report](https://scikit-learn.org/stable/modules/generated/sklearn.metrics.classification_report.html) on the test set.
2. Train logistic regression with average embedding representation extracted using spaCy, calculate the train and test accuracy and show the [classification report](https://scikit-learn.org/stable/modules/generated/sklearn.metrics.classification_report.html) on the test set.

In [109]:
df = pd.read_csv("data/cleaned_hm.csv", index_col=0)
sample_df = df.dropna()
sample_df.head()

,wid,reflection_period,original_hm,cleaned_hm,modified,num_sentence,ground_truth_category,predicted_category
hmid,,,,,,,,
27676,206,24h,We had a serious talk with some friends of our...,We had a serious talk with some friends of our...,True,2,bonding,bonding
27678,45,24h,I meditated last night.,I meditated last night.,True,1,leisure,leisure
27697,498,24h,My grandmother start to walk from the bed afte...,My grandmother start to walk from the bed afte...,True,1,affection,affection
27705,5732,24h,I picked my daughter up from the airport and w...,I picked my daughter up from the airport and w...,True,1,bonding,affection
27715,2272,24h,when i received flowers from my best friend,when i received flowers from my best friend,True,1,bonding,bonding


In [113]:
sample_df = sample_df.rename(
    columns={"cleaned_hm": "moment", "ground_truth_category": "target"}
)

In [114]:
train_df, test_df = train_test_split(sample_df, test_size=0.3, random_state=123)
X_train, y_train = train_df["moment"], train_df["target"]
X_test, y_test = test_df["moment"], test_df["target"]

You need `spacy` to run the code below. If it's not in your course conda environment, you need to install it. 

> conda install -c conda-forge spacy

You also need to download the following language model. 

> python -m spacy download en_core_web_md

In [110]:
import spacy
import en_core_web_md

# nlp = spacy.load("en_core_web_md")
nlp = en_core_web_md.load()

<div class="alert alert-warning">

Solution_1.5
    
</div>

In [120]:
from sklearn.metrics import classification_report


In [121]:
bow_train_accuracy = None
bow_test_accuracy = None
bow_report = None # use digits=2

# max_iter was set to 1000 to avoid getting warning that the "number of iterations reached limit".
pipe = make_pipeline(CountVectorizer(stop_words="english"), LogisticRegression(max_iter=1000))
pipe.fit(X_train, y_train)

bow_train_accuracy = pipe.score(X_train, y_train)
bow_test_accuracy = pipe.score(X_test, y_test)

bow_pred = pipe.predict(X_test)
bow_report = classification_report(y_test, bow_pred, digits=2)

print("Train accuracy: %0.2f" % bow_train_accuracy)
print("Test accuracy: %0.2f" % bow_test_accuracy) 
print("Classification report: \n", bow_report)

train accuracy: 0.96
test accuracy: 0.82
classification report: 
                   precision    recall  f1-score   support

     achievement       0.79      0.87      0.83      1302
       affection       0.90      0.91      0.91      1423
         bonding       0.91      0.85      0.88       492
enjoy_the_moment       0.60      0.54      0.57       469
        exercise       0.91      0.57      0.70        74
         leisure       0.73      0.70      0.71       407
          nature       0.73      0.46      0.57        71

        accuracy                           0.82      4238
       macro avg       0.80      0.70      0.74      4238
    weighted avg       0.82      0.82      0.81      4238



In [117]:
X_train_embeddings = None
X_test_embeddings = None

# adapted from piazza @599_f1
X_train_embeddings = pd.DataFrame([text.vector for text in nlp.pipe(X_train)])
X_test_embeddings = pd.DataFrame([text.vector for text in nlp.pipe(X_test)])

In [118]:
X_train_embeddings.head()

,0,1,2,3,4,5,6,7,8,9,...,290,291,292,293,294,295,296,297,298,299
0,-0.690874,-0.755216,-3.279650,-1.111956,2.333812,1.712701,0.346985,5.052109,-1.078272,1.564646,...,0.978959,0.228331,0.843013,-1.340969,-0.884684,1.971317,-0.880458,-0.599687,-5.299727,-0.139515
1,0.021659,-1.051312,-4.063361,-1.818072,3.027572,-1.226410,0.035204,2.806751,-0.181556,1.530708,...,0.873182,-1.440154,0.219029,-1.426304,-1.591761,-0.475234,-0.122053,0.695197,-4.421231,-0.027261
2,-0.101639,-1.674547,-1.391433,-3.230565,0.636887,0.756545,0.205278,5.351284,-2.872737,3.033967,...,0.594443,0.435623,-0.556652,-2.759545,-1.521208,0.728254,-0.365033,1.230975,-5.989079,0.142662
3,-1.502830,-0.245505,-2.919757,-1.312146,3.315014,-0.281209,1.019417,3.909962,-0.440415,0.797455,...,0.452126,-0.261931,-0.375499,-0.218783,-1.355882,0.384430,0.619531,0.548577,-3.217188,0.119259
4,-0.503838,-0.450076,-4.626111,-2.820389,4.793096,-0.805586,1.518527,5.712772,1.548753,1.589522,...,-0.293538,-0.217686,-3.700689,-1.188137,0.648456,-2.548641,-0.413536,1.096762,-3.927443,-1.008204


In [119]:
X_test_embeddings.head()

,0,1,2,3,4,5,6,7,8,9,...,290,291,292,293,294,295,296,297,298,299
0,-0.958416,3.125293,-4.034988,-2.041907,2.595106,0.445679,0.386465,5.083802,-1.852383,1.560549,...,-0.379911,-0.726037,-1.127668,-0.563859,0.177253,-0.294613,-0.855359,1.358393,-7.270348,1.399086
1,-1.299894,0.630366,-2.318005,-2.507104,4.050345,-0.310809,0.272773,3.979269,1.976808,0.731067,...,0.931720,0.025334,-0.485929,-0.743087,-1.798334,2.132924,0.299637,0.670521,-3.445617,0.822201
2,1.671838,-3.753560,-2.243932,-6.034901,-2.993384,2.103476,-0.182704,1.314220,0.393286,0.491056,...,-0.021817,0.214875,-1.920058,-0.585280,-2.042112,1.066200,-0.503640,0.496954,-0.952958,-0.096886
3,3.285316,-1.347252,-5.520160,-1.793326,-0.319224,-4.395940,-0.787140,3.501826,-2.966559,4.460822,...,2.226200,-1.617096,1.458074,-4.914721,-2.545942,-3.723066,-2.130358,1.875884,-8.811860,-0.704100
4,-3.296173,1.518396,-3.726718,-1.118125,1.690331,-1.578362,0.601601,5.086967,-0.884238,3.076164,...,0.435902,0.211110,1.393579,0.039486,-2.776130,1.652559,3.089273,2.418594,-3.963638,0.478087


In [124]:
emb_train_accuracy = None
emb_test_accuracy = None
emb_report = None # use digits=2

# max_iter was set to 10000 to avoid getting warning that the "number of iterations reached limit".
emb_pipe = make_pipeline(LogisticRegression(max_iter=10000))
emb_pipe.fit(X_train_embeddings, y_train)


emb_train_accuracy = emb_pipe.score(X_train_embeddings, y_train)
emb_test_accuracy = emb_pipe.score(X_test_embeddings, y_test)

emb_pred = emb_pipe.predict(X_test_embeddings)
emb_report = classification_report(y_test, emb_pred, digits=2)


print("Train accuracy with embeddings: %.2f" % emb_train_accuracy)
print("Test accuracy with embeddings: %.2f" % emb_test_accuracy)
print("Classification report with embeddings:\n", emb_report)


Train accuracy with embeddings: 0.85
Test accuracy with embeddings: 0.79
Classification report with embeddings:
                   precision    recall  f1-score   support

     achievement       0.81      0.83      0.82      1302
       affection       0.86      0.91      0.89      1423
         bonding       0.83      0.77      0.80       492
enjoy_the_moment       0.57      0.51      0.53       469
        exercise       0.68      0.76      0.72        74
         leisure       0.72      0.65      0.68       407
          nature       0.68      0.72      0.70        71

        accuracy                           0.79      4238
       macro avg       0.74      0.74      0.73      4238
    weighted avg       0.79      0.79      0.79      4238



<br><br>

<!-- BEGIN QUESTION -->

### 1.6 Discussion

_Points:_ 6

**Your tasks:**
1. Briefly explain the difference between using `CountVectorizer` vs. average-embedding approach for text classification.  
2. Which representation among these two would be more interpretable? Why?   
3. Are we using any transfer learning here? If yes, are you observing any benefits of transfer learning? Briefly discuss. 

<div class="alert alert-warning">

Solution_1.6
    
</div>

1. CountVectorizer counts the number of occurances of each word in the provided text, representing the frequency of that word in the data. Word order and context are not considered when using CountVectorizer. The average-embeddings approach captures semantic contextual information for each word in addition to frequency counts based on that word's embeddings. 

2. With both methods, we are able to examine the report to get a general understanding of how both method performed. However, CountVectorizer is easier to interpret because it is a simplified analysis in comparison to using average-embeddings. With CountVectorizer, we can easily see which words contribute more or less to classification decisions since each feature corresponds to a single unique word in the data set. This information is more obscured and complex when we use average-embeddings because context is included in the analysis.

3. We are using transfer learning because we are leveraging spaCy's pre-trained word embeddings. We are seeing the benefits here because we aren't having to train our model on a huge data set. Since we are able to utilize spaCy's pre-training word embeddings, our model can leverage the knowledge learned from this large data set and potentially capture more nuanced semantic relationships that may have otherwise been missed if we had simply trained out model on our relatively small single dataset.

<!-- END QUESTION -->

<br><br><br><br>

## Exercise 2: Topic modeling 

The overarching goal of topic modeling is understanding high-level themes in a large collection of texts in an unsupervised way. 

In this exercise you will explore topics in a subset of `scikit-learn`'s [20 newsgroups text dataset](https://scikit-learn.org/0.19/datasets/twenty_newsgroups.html) using `scikit-learn`'s `LatentDirichletAllocation` (LDA) model. 

Usually, topic modeling is used for discovering abstract "topics" that occur in a collection of documents when you do not know the actual topics present in the documents. But 20 newsgroups text dataset is labeled with categories (e.g., sports, hardware, religion), and you will be able to cross-check the topics discovered by your model with these available topics. 

The starter code below loads the train and test portion of the data and convert the train portion into a pandas DataFrame. For speed, we will only consider documents with the following 8 categories. 

In [125]:
from sklearn.datasets import fetch_20newsgroups

In [126]:
cats = [
    "rec.sport.hockey",
    "rec.sport.baseball",
    "soc.religion.christian",
    "alt.atheism",
    "comp.graphics",
    "comp.windows.x",
    "talk.politics.mideast",
    "talk.politics.guns",
]  # We'll only consider these categories out of 20 categories for speed.

newsgroups_train = fetch_20newsgroups(
    subset="train", remove=("headers", "footers", "quotes"), categories=cats
)
X_news_train, y_news_train = newsgroups_train.data, newsgroups_train.target
df = pd.DataFrame(X_news_train, columns=["text"])
df["target"] = y_news_train
df["target_name"] = [
    newsgroups_train.target_names[target] for target in newsgroups_train.target
]
df

,text,target,target_name
0,"You know, I was reading 18 U.S.C. 922 and some...",6,talk.politics.guns
1,\n\n\nIt's not a bad question: I don't have an...,1,comp.graphics
2,"\nActuallay I don't, but on the other hand I d...",1,comp.graphics
3,"The following problem is really bugging me,\na...",2,comp.windows.x
4,\n\n This is the latest from UPI \n\n For...,7,talk.politics.mideast
...,...,...,...
4558,Hi Everyone ::\n\nI am looking for some soft...,1,comp.graphics
4559,Archive-name: x-faq/part3\nLast-modified: 1993...,2,comp.windows.x
4560,"\nThat's nice, but it doesn't answer the quest...",6,talk.politics.guns
4561,"Hi,\n I just got myself a Gateway 4DX-33V ...",2,comp.windows.x


In [127]:
newsgroups_train.target_names

['alt.atheism',
 'comp.graphics',
 'comp.windows.x',
 'rec.sport.baseball',
 'rec.sport.hockey',
 'soc.religion.christian',
 'talk.politics.guns',
 'talk.politics.mideast']

<br><br>

<!-- BEGIN QUESTION -->

### 2.1 Preprocessing using [spaCy](https://spacy.io/)

_Points:_ 8

In this exercise you'll prepare the data for topic modeling using [spaCy](https://spacy.io/). Preprocessing is a crucial step before training an LDA model and it markedly affects topic modeling results. So let's carry out preprocessing. 

**Your tasks:** 

Write code to carry out preprocessing of the "text" column in the dataframe above and store the preprocessed text in a new column called "text_pp" in the dataframe. 


Note that there is no such thing as "perfect" preprocessing. You'll have to make your own judgments and decisions on which tokens are more informative and which ones are less informative for the given task. Some common text preprocessing steps for topic modeling are: 
- getting rid of slashes or other weird characters
- sentence segmentation and tokenization      
- getting rid of urls and email addresses
- getting rid of other fairly unique tokens which are not going to help us in topic modeling  
- excluding stopwords and punctuation 
- lemmatization        

You might have to go back and forth between the preprocessing and topic modeling and interpretation steps in the next exercises. 

> Check out [these available attributes](https://spacy.io/api/token#attributes) for `token` in spaCy which might help you with preprocessing. 

> You can also get rid of words with specific POS tags. [Here](https://spacy.io/api/annotation/#pos-en) is the list of part-of-speech tags used in spaCy. 

> Note that preprocessing the corpus might take some time. So here are a couple of suggestions: 1) During the debugging phase, work on a smaller subset of the data. 2) Once you're done with the preprocessing part, you might want to save the preprocessed data so that you don't run the preprocessing part every time you run the notebook. 


<div class="alert alert-warning">

Solution_2_1
    
</div>

In [136]:
def preprocess_text(text):
    doc = nlp(text)

    # Convert text to lowercase
    preprocessed_text = [token.text.lower() for token in doc]

    # Remove stopwords, punctuation, and short words
    preprocessed_text = [token for token in preprocessed_text if token not in nlp.Defaults.stop_words and len(token) > 2 and token.isalpha()]
   
    # Remove numerical values
    preprocessed_text = [token for token in preprocessed_text if not token.isdigit()]
    
    # Remove non-alphanumeric characters
    preprocessed_text = [token for token in preprocessed_text if token.isalnum()]
    
    # Lemmatize
    preprocessed_text = [token.lemma_ for token in nlp(' '.join(preprocessed_text))]
    
    # Join the tokens back into a single string
    preprocessed_text = ' '.join(preprocessed_text)

    return preprocessed_text


In [137]:
df['text_pp'] = df['text'].apply(preprocess_text)

df.head()

,text,target,target_name,text_pp
0,"You know, I was reading 18 U.S.C. 922 and some...",6,talk.politics.guns,know read sence wonder help provide paragraph ...
1,\n\n\nIt's not a bad question: I don't have an...,1,comp.graphics,bad question ref list algorithm think bit hard...
2,"\nActuallay I don't, but on the other hand I d...",1,comp.graphics,actuallay hand support idea have newsgroup asp...
3,"The following problem is really bugging me,\na...",2,comp.windows.x,follow problem bug appreciate help create wind...
4,\n\n This is the latest from UPI \n\n For...,7,talk.politics.mideast,late upi foreign ministry spokesman ferhat ata...


<!-- END QUESTION -->

<br><br>

<!-- BEGIN QUESTION -->

### 2.2 Justification

_Points:_ 2

**Your tasks:**

Outline the preprocessing steps you carried out in the previous exercise and provide a brief justification for these steps.

<div class="alert alert-warning">

Solution_2_2
    
</div>

1. Text was tokenized to split it into individual tokens to allow for further processing.
2. The text was converted to lowercase to ensure consistency and prevent duplication of words due to different cases. This helps reduce dimensionality in the data.
3. Stopwords, punctuation, and short words were removed from the text because they don't typically contribute to the semantic meaning of the text.
4. Numberical values were removed because these are generally unimportant for topic modelling.
5. Non-alphanumeric characters were removed because these also typically do not contribute to the semantic meaning of the text.
6. Tokens were lemmatized to normalize the text by converting words to their base form. This allows words like "cars" and "car" to be treated semantically identically while also reducing the dimensionality of the data.
7. The tokens were joined back together into a single string so they could be added back into the df and be used in further modelling and analysis.


<!-- END QUESTION -->

<br><br>

<!-- BEGIN QUESTION -->

### 2.3 Build a topic model using sklearn's LatentDirichletAllocation

_Points:_ 4

**Your tasks:**
1. Create a topic model on the preprocessed data using [sklearn's `LatentDirichletAllocation`](https://scikit-learn.org/stable/modules/generated/sklearn.decomposition.LatentDirichletAllocation.html). Pick a reasonable number for `n_components`, i.e., number of topics and briefly justify your choice.

<div class="alert alert-warning">

Solution_2_3
    
</div>

There are 4 general categories in our topics: religion, sports, politics, technology.

In [142]:
from sklearn.decomposition import LatentDirichletAllocation

n_topics = 4

vec = CountVectorizer(stop_words='english')
text_fit_trans = vec.fit_transform(df["text_pp"])

lda = LatentDirichletAllocation(n_components=n_topics, random_state=42)
lda.fit(text_fit_trans)

topics = lda.transform(text_fit_trans)

<!-- END QUESTION -->

<br><br>

<!-- BEGIN QUESTION -->

### 2.4 Exploring word topic association

_Points:_ 5

**Your tasks:**
1. Show top 10 words for each of your topics and suggest labels for each of the topics (similar to how we came up with labels "health and nutrition", "fashion", and "machine learning" in the toy example we saw in class). 

> If your topics do not make much sense, you might have to go back to preprocessing in Exercise 2.1, improve it, and train your LDA model again. 

<div class="alert alert-warning">

Solution_2_4
    
</div>

In [143]:

feature_names = vec.get_feature_names_out()
n_top_words = 10  

for topic_idx, topic in enumerate(lda.components_):
    print(f"Topic {topic_idx}:")
    top_words_indices = topic.argsort()[:-n_top_words - 1:-1]
    top_words = [feature_names[i] for i in top_words_indices]
    print(", ".join(top_words))
    print()


Topic 0:
file, program, use, image, window, include, available, server, run, work

Topic 1:
game, team, year, play, good, player, win, season, think, time

Topic 2:
people, armenian, gun, right, state, israel, turkish, government, jews, kill

Topic 3:
god, people, say, know, think, come, believe, time, thing, like



Suggested topic labels:
- 0 : Computers and Software
- 1 : Sports
- 2 : Political Conflict
- 3 : Religion and Philosophy


<!-- END QUESTION -->

<br><br>

<!-- BEGIN QUESTION -->

### 2.5 Exploring document topic association

_Points:_ 5

**Your tasks:**
1. Show the document topic assignment of the first five documents from `df`. 
2. Comment on the document topic assignment of the model. 

<div class="alert alert-warning">

Solution_2_5
    
</div>

<!-- END QUESTION -->

<br><br><br><br>

<!-- BEGIN QUESTION -->

## Exercise 3: Short answer questions 
<hr>

_Points:_ 6

1. In lecture 18, we talked about multi-class classification. Comment on how each model in the list below might be handling multiclass classification. Check `scikit-learn` documentation for each of these models when you answer this question.  
    - Decision Tree
    - KNN
    - Random Forest    
    - Logistic Regression
    - SVM RBF
2. What is transfer learning in natural language processing or computer vision? Briefly explain.     
3. In Lecture 18 we briefly discussed how neural networks are sort of like `Pipeline`s, in the sense that they involve multiple sequential transformations of the data, finally resulting in the prediction. Why was this property useful when it came to transfer learning?

<div class="alert alert-warning">

Solution_3
    
</div>

_Type your answer here, replacing this text._

<!-- END QUESTION -->

<br><br><br><br>

**PLEASE READ BEFORE YOU SUBMIT:** When you are ready to submit your assignment do the following:

1. Run all cells in your notebook to make sure there are no errors by doing `Kernel -> Restart Kernel and Clear All Outputs` and then `Run -> Run All Cells`. 
2. Notebooks with cell execution numbers out of order or not starting from “1” will have marks deducted. Notebooks without the output displayed may not be graded at all (because we need to see the output in order to grade your work).
3. Upload the assignment using [PrairieLearn](https://ca.prairielearn.com/pl/course_instance/6697). Don't forget to rename your file `hw6_sol.ipynb`.

![](img/eva-well-done.png)